In [19]:
import os
import glob
import cv2
import random
import pandas as pd
from skimage import io
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import h5py

# Network building stuff
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics
import torch.distributions as dist

In [5]:
import sys
# sys.path.append("/home2/sdokania/all_projects/project-noisypixel/")

In [6]:
from src.models import *
from src.dataset.dataloader import OccupancyNetDatasetHDF
from src.trainer import ONetLit
from src.utils import Config, count_parameters

In [7]:
config = Config()
config.data_root = "/ssd_scratch/cvit/sdokania/processed_data/hdf_data/"
config.batch_size = 32

Setting sexperiment path as : /home2/sdokania/all_projects/occ_artifacts/initial


In [8]:

vars(config)

{'c_dim': 128,
 'h_dim': 128,
 'p_dim': 3,
 'data_root': '/ssd_scratch/cvit/sdokania/processed_data/hdf_data/',
 'batch_size': 32,
 'output_dir': '/home2/sdokania/all_projects/occ_artifacts/',
 'exp_name': 'initial',
 'encoder': 'efficientnet-b0',
 'decoder': 'decoder-cbn',
 'exp_path': '/home2/sdokania/all_projects/occ_artifacts/initial',
 'lr': 0.0003}

In [9]:
onet = ONetLit(config)

Loaded pretrained weights for efficientnet-b0


In [11]:
net = ONetLit.load_from_checkpoint("../occ_artifacts/efficient_cbn_bs_64_full_data/lightning_logs/version_1/checkpoints/epoch=28-step=13919.ckpt", cfg=config)

Loaded pretrained weights for efficientnet-b0


In [12]:
dataset = OccupancyNetDatasetHDF(config.data_root, mode="val", num_points=10000)

In [13]:
test_img, test_pts, test_gt = dataset[0]

In [14]:
def make_3d_grid(bb_min, bb_max, shape):
    ''' Makes a 3D grid.
    Args:
        bb_min (tuple): bounding box minimum
        bb_max (tuple): bounding box maximum
        shape (tuple): output shape
    '''
    size = shape[0] * shape[1] * shape[2]

    pxs = torch.linspace(bb_min[0], bb_max[0], shape[0])
    pys = torch.linspace(bb_min[1], bb_max[1], shape[1])
    pzs = torch.linspace(bb_min[2], bb_max[2], shape[2])

    pxs = pxs.view(-1, 1, 1).expand(*shape).contiguous().view(size)
    pys = pys.view(1, -1, 1).expand(*shape).contiguous().view(size)
    pzs = pzs.view(1, 1, -1).expand(*shape).contiguous().view(size)
    p = torch.stack([pxs, pys, pzs], dim=1)

    return p

In [21]:
vg = make_3d_grid((-0.5,)*3, (0.5,)*3, (32,)*3)
vg.shape

torch.Size([32768, 3])

In [22]:
out = net(test_img.unsqueeze(0), vg.unsqueeze(0))

In [23]:
c = net.net.encoder(test_img.unsqueeze(0)).detach()

In [24]:
p = dist.Bernoulli(logits=out)
vg[p.probs.flatten() > 0.5].shape

torch.Size([621, 3])

In [25]:
good_p = vg[p.probs.flatten() > 0.5]
good_p.requires_grad_()

tensor([[-0.4032, -0.0161, -0.0161],
        [-0.4032, -0.0161,  0.0161],
        [-0.4032, -0.0161,  0.0484],
        ...,
        [ 0.3387, -0.0161,  0.0484],
        [ 0.3710, -0.0161, -0.0161],
        [ 0.3710, -0.0161,  0.0161]], requires_grad=True)

In [26]:
net.zero_grad()
outs = net.net.decoder(good_p.unsqueeze(0), c)
outs = outs.sum()
outs.backward()

In [29]:
 ni = -good_p.grad
ni = ni / torch.norm(ni, dim=-1, keepdim=True)
ni = ni.squeeze(0).cpu().numpy()
print(ni.shape)

(621, 3)


In [32]:
np.save("../point_vals", good_p.detach().numpy())
np.save("../normal_vals", ni)

In [10]:
onet(torch.randn(1, 3, 224, 224), torch.randn(1, 1024, 3)).shape

torch.Size([1, 1024])

In [5]:
TensorBoardLogger?

Init signature:
TensorBoardLogger(
    save_dir: str,
    name: Union[str, NoneType] = 'default',
    version: Union[int, str, NoneType] = None,
    log_graph: bool = False,
    default_hp_metric: bool = True,
    prefix: str = '',
    **kwargs,
)
Docstring:     
Log to local file system in `TensorBoard <https://www.tensorflow.org/tensorboard>`_ format.

Implemented using :class:`~torch.utils.tensorboard.SummaryWriter`. Logs are saved to
``os.path.join(save_dir, name, version)``. This is the default logger in Lightning, it comes
preinstalled.

Example:
    >>> from pytorch_lightning import Trainer
    >>> from pytorch_lightning.loggers import TensorBoardLogger
    >>> logger = TensorBoardLogger("tb_logs", name="my_model")
    >>> trainer = Trainer(logger=logger)

Args:
    save_dir: Save directory
    name: Experiment name. Defaults to ``'default'``. If it is the empty string then no per-experiment
        subdirectory is used.
    version: Experiment version. If version is not specifi

In [13]:
# Define the trainer object
trainer = pl.Trainer(
    gpus=1,
    # auto_scale_batch_size='binsearch',
    logger=logger,
    min_epochs=1,
    max_epochs=1,
    default_root_dir=config.output_dir,
    log_every_n_steps=10,
    progress_bar_refresh_rate=5,
    # precision=16,
    # stochastic_weight_avg=True,
    # track_grad_norm=2,
    callbacks=[checkpoint_callback],
    check_val_every_n_epoch=1,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [14]:
# Start training
trainer.fit(onet)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type      | Params
-----------------------------------
0 | net  | OccNetImg | 4.7 M 
-----------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.802    Total estimated model params size (MB)
/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [7]:
vd = OccupancyNetDatasetHDF(config.data_root, mode="val")
vdl = torch.utils.data.DataLoader(vd, batch_size=100, shuffle=False, num_workers=8)

In [8]:
for ix in vdl:
    print(ix[0].shape, ix[1].shape, ix[2].shape)

torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100,

In [42]:
len(vdl)

44

In [13]:
fname = "/ssd_scratch/cvit/sdokania/hdf_data/hdf_data/04256520_bdfcf2086fafb0fec8a04932b17782af.h5"
hf = h5py.File(fname, 'r')

In [15]:
hf.keys()

<KeysViewHDF5 ['camera', 'images']>